<a href="https://colab.research.google.com/github/Josecp03/Google-Colab/blob/main/PastorIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy gtts SpeechRecognition pydub openpyxl --quiet
!python -m spacy download es_core_news_md --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 18.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import spacy
import pandas as pd
import numpy as np
import re
from gtts import gTTS
import speech_recognition as sr
from IPython.display import Audio, display, HTML
import os
import json
from collections import defaultdict
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Cargar modelo de spaCy
nlp = spacy.load("es_core_news_md")

print("✅ Modelo de spaCy cargado correctamente")
print(f"📚 Modelo: {nlp.meta['name']} - Versión: {nlp.meta['version']}")

✅ Modelo de spaCy cargado correctamente
📚 Modelo: core_news_md - Versión: 3.8.0


In [4]:
df_ovejas = pd.read_excel('sample_data/dataset_500_ovejas.xlsx', sheet_name='Ovejas')

print("="*60)
print("📊 DATASET DE 500 OVEJAS CARGADO")
print("="*60)
print(f"\n🐑 Total de ovejas: {len(df_ovejas)}")
print(f"\n🏷️  Columnas disponibles: {list(df_ovejas.columns)}")
print(f"\n🐏 Distribución por raza:")
print(df_ovejas['raza'].value_counts())
print("\n" + "="*60)
print("📋 Primeras 5 ovejas del dataset:")
print("="*60)
display(df_ovejas.head())

# Añadir ID único a cada oveja
df_ovejas['id_oveja'] = [f"OV{str(i+1).zfill(4)}" for i in range(len(df_ovejas))]

print(f"\n✅ Dataset preparado con {len(df_ovejas)} ovejas identificadas")

📊 DATASET DE 500 OVEJAS CARGADO

🐑 Total de ovejas: 500

🏷️  Columnas disponibles: ['raza', 'peso_kg', 'lana_color', 'lana_longitud_cm', 'orejas', 'cuernos', 'altura_cm']

🐏 Distribución por raza:
raza
Corriedale    120
Manchega      117
Merina         98
Suffolk        92
Dorper         73
Name: count, dtype: int64

📋 Primeras 5 ovejas del dataset:


,raza,peso_kg,lana_color,lana_longitud_cm,orejas,cuernos,altura_cm
0,Merina,48.5,blanca,9.8,medianas,sin cuernos,63.6
1,Merina,51.0,blanca,8.8,caídas,sin cuernos,64.1
2,Merina,41.8,blanca,6.4,caídas,pequeños,58.7
3,Merina,44.8,blanca,9.2,caídas,pequeños,63.9
4,Merina,41.8,blanca,9.1,medianas,pequeños,63.3



✅ Dataset preparado con 500 ovejas identificadas


In [5]:
# ============================================
# ESTADO INICIAL DEL REBAÑO
# ============================================
# Definimos las ubicaciones posibles y distribuimos las ovejas

# Ubicaciones disponibles
UBICACIONES = [
    "corral norte",
    "corral sur",
    "pradera este",
    "pradera oeste",
    "nave",
    "cobertizo",
    "ordeño"
]

# Añadir columna de ubicación inicial (todas empiezan en pradera este)
df_ovejas['ubicacion'] = 'pradera este'

# Crear estructura del estado del rebaño
def obtener_estado_rebano():
    """Retorna el estado actual del rebaño agrupado por ubicación y raza"""
    estado = {}
    for ubicacion in UBICACIONES:
        estado[ubicacion] = {}
        for raza in df_ovejas['raza'].unique():
            cantidad = len(df_ovejas[(df_ovejas['ubicacion'] == ubicacion) &
                                     (df_ovejas['raza'] == raza)])
            if cantidad > 0:
                estado[ubicacion][raza] = cantidad
    return estado

# Función para mostrar el estado actual
def mostrar_estado():
    """Muestra el estado actual del rebaño de forma visual"""
    print("\n" + "="*70)
    print("🐑 ESTADO ACTUAL DEL REBAÑO")
    print("="*70)

    estado = obtener_estado_rebano()
    total_general = 0

    for ubicacion, razas in estado.items():
        if razas:  # Solo mostrar ubicaciones con ovejas
            total_ubicacion = sum(razas.values())
            total_general += total_ubicacion
            print(f"\n📍 {ubicacion.upper()}: {total_ubicacion} ovejas")
            for raza, cantidad in razas.items():
                print(f"   • {raza}: {cantidad}")

    print(f"\n{'='*70}")
    print(f"🐏 TOTAL REBAÑO: {total_general} ovejas")
    print("="*70)

# Mostrar estado inicial
print("✅ Sistema de estado del rebaño inicializado")
mostrar_estado()


✅ Sistema de estado del rebaño inicializado

🐑 ESTADO ACTUAL DEL REBAÑO

📍 PRADERA ESTE: 500 ovejas
   • Merina: 98
   • Manchega: 117
   • Suffolk: 92
   • Dorper: 73
   • Corriedale: 120

🐏 TOTAL REBAÑO: 500 ovejas


In [7]:
# ============================================
# INTERJECCIONES Y COMANDOS DE PASTOR
# ============================================
# Diccionario de sonidos tradicionales del pastoreo español

INTERJECCIONES = {
    "agrupar": [
        "¡Ea, ea, eaaa!",
        "¡Vamos, juntarse!",
        "¡Arre, arre, todos juntos!",
        "¡Ehhh, aquí, aquí!"
    ],
    "mover": [
        "¡Arreeee!",
        "¡Vamos, vamos!",
        "¡Ea, pa'lante!",
        "¡Arre, arre!"
    ],
    "separar": [
        "¡Eh, eh, pss pss!",
        "¡Chu chu chu!",
        "¡Pss, fuera, fuera!",
        "¡Ehhh, atrás!"
    ],
    "llamar": [
        "¡Vamos, vamos!",
        "¡Oveee, oveee!",
        "¡Aquí, aquí!",
        "¡Ea, ea!"
    ],
    "detener": [
        "¡Eehhhh!",
        "¡Quietas!",
        "¡Alto ahí!",
        "¡Sooo!"
    ],
    "llevar": [
        "¡Arre hacia allá!",
        "¡Vamos pa'llá!",
        "¡Ea, pa'l corral!",
        "¡Adelante!"
    ]
}

# Función para obtener interjección según acción
def obtener_interjeccion(accion, destino=None):
    """
    Retorna una interjección apropiada según la acción
    Si hay destino, se personaliza
    """
    accion_base = accion.lower()

    # Mapear variaciones de acciones
    if any(palabra in accion_base for palabra in ["agrupar", "juntar", "reunir"]):
        interjeccion = np.random.choice(INTERJECCIONES["agrupar"])
    elif any(palabra in accion_base for palabra in ["mover", "llevar", "trasladar"]):
        interjeccion = np.random.choice(INTERJECCIONES["llevar"])
        if destino:
            interjeccion += f" ¡Hacia {destino}!"
    elif any(palabra in accion_base for palabra in ["separar", "apartar"]):
        interjeccion = np.random.choice(INTERJECCIONES["separar"])
    elif any(palabra in accion_base for palabra in ["llamar", "traer"]):
        interjeccion = np.random.choice(INTERJECCIONES["llamar"])
    elif any(palabra in accion_base for palabra in ["detener", "parar", "quieto"]):
        interjeccion = np.random.choice(INTERJECCIONES["detener"])
    else:
        interjeccion = "¡Ea, ea!"

    return interjeccion

print("✅ Diccionario de interjecciones de pastor cargado")
print(f"📢 Acciones disponibles: {list(INTERJECCIONES.keys())}")
print(f"\n🔊 Ejemplo de interjección para 'agrupar': {obtener_interjeccion('agrupar')}")
print(f"🔊 Ejemplo de interjección para 'mover': {obtener_interjeccion('mover', 'corral norte')}")


✅ Diccionario de interjecciones de pastor cargado
📢 Acciones disponibles: ['agrupar', 'mover', 'separar', 'llamar', 'detener', 'llevar']

🔊 Ejemplo de interjección para 'agrupar': ¡Ehhh, aquí, aquí!
🔊 Ejemplo de interjección para 'mover': ¡Adelante! ¡Hacia corral norte!


In [8]:
# ============================================
# ENTITY RULER - RECONOCIMIENTO DE ENTIDADES
# ============================================
# Configuramos patrones para identificar razas, ubicaciones y acciones

# Limpiar ruler si existe
if "entity_ruler" in nlp.pipe_names:
    nlp.remove_pipe("entity_ruler")

# Crear EntityRuler
ruler = nlp.add_pipe("entity_ruler", before="ner")

# Patrones para RAZAS
patrones_razas = [
    {"label": "RAZA", "pattern": "Merina"},
    {"label": "RAZA", "pattern": "merina"},
    {"label": "RAZA", "pattern": "merinas"},
    {"label": "RAZA", "pattern": "Manchega"},
    {"label": "RAZA", "pattern": "manchega"},
    {"label": "RAZA", "pattern": "manchegas"},
    {"label": "RAZA", "pattern": "Suffolk"},
    {"label": "RAZA", "pattern": "suffolk"},
    {"label": "RAZA", "pattern": "Dorper"},
    {"label": "RAZA", "pattern": "dorper"},
    {"label": "RAZA", "pattern": "dorpers"},
    {"label": "RAZA", "pattern": "Corriedale"},
    {"label": "RAZA", "pattern": "corriedale"},
    {"label": "RAZA", "pattern": "corriedales"},
]

# Patrones para UBICACIONES
patrones_ubicaciones = [
    {"label": "UBICACION", "pattern": [{"LOWER": "corral"}, {"LOWER": "norte"}]},
    {"label": "UBICACION", "pattern": [{"LOWER": "corral"}, {"LOWER": "sur"}]},
    {"label": "UBICACION", "pattern": [{"LOWER": "pradera"}, {"LOWER": "este"}]},
    {"label": "UBICACION", "pattern": [{"LOWER": "pradera"}, {"LOWER": "oeste"}]},
    {"label": "UBICACION", "pattern": "nave"},
    {"label": "UBICACION", "pattern": "cobertizo"},
    {"label": "UBICACION", "pattern": "ordeño"},
    {"label": "UBICACION", "pattern": "corral"},
    {"label": "UBICACION", "pattern": "pradera"},
]

# Patrones para ACCIONES
patrones_acciones = [
    {"label": "ACCION", "pattern": "agrupa"},
    {"label": "ACCION", "pattern": "agrupar"},
    {"label": "ACCION", "pattern": "junta"},
    {"label": "ACCION", "pattern": "juntar"},
    {"label": "ACCION", "pattern": "reunir"},
    {"label": "ACCION", "pattern": "mueve"},
    {"label": "ACCION", "pattern": "mover"},
    {"label": "ACCION", "pattern": "lleva"},
    {"label": "ACCION", "pattern": "llevar"},
    {"label": "ACCION", "pattern": "traslada"},
    {"label": "ACCION", "pattern": "trasladar"},
    {"label": "ACCION", "pattern": "separa"},
    {"label": "ACCION", "pattern": "separar"},
    {"label": "ACCION", "pattern": "aparta"},
    {"label": "ACCION", "pattern": "apartar"},
]

# Añadir todos los patrones al ruler
ruler.add_patterns(patrones_razas + patrones_ubicaciones + patrones_acciones)

print("✅ EntityRuler configurado correctamente")
print(f"📝 Patrones añadidos:")
print(f"   • Razas: {len(patrones_razas)}")
print(f"   • Ubicaciones: {len(patrones_ubicaciones)}")
print(f"   • Acciones: {len(patrones_acciones)}")
print(f"   • TOTAL: {len(patrones_razas + patrones_ubicaciones + patrones_acciones)} patrones")

# Prueba rápida
texto_prueba = "Mueve las ovejas Merina al corral norte"
doc = nlp(texto_prueba)
print(f"\n🧪 Prueba con: '{texto_prueba}'")
print(f"🎯 Entidades detectadas:")
for ent in doc.ents:
    print(f"   • {ent.text} → {ent.label_}")


✅ EntityRuler configurado correctamente
📝 Patrones añadidos:
   • Razas: 14
   • Ubicaciones: 9
   • Acciones: 15
   • TOTAL: 38 patrones

🧪 Prueba con: 'Mueve las ovejas Merina al corral norte'
🎯 Entidades detectadas:
   • Merina → RAZA
   • corral norte → UBICACION


In [9]:
# ============================================
# EXTRACCIÓN DE ENTIDADES Y RELACIONES (NER + RE)
# ============================================

from spacy.matcher import Matcher

# Crear Matcher para patrones de dependencias
matcher = Matcher(nlp.vocab)

# Patrones para detectar estructura completa de órdenes
# Patrón: [ACCION] + [CANTIDAD?] + [RAZA?] + [ORIGEN?] + [DESTINO?]

# Función principal de extracción
def extraer_informacion(texto):
    """
    Extrae entidades y relaciones de una orden de pastoreo
    Retorna: diccionario con acción, raza, cantidad, origen, destino
    """
    doc = nlp(texto.lower())

    # Inicializar resultado
    info = {
        "texto_original": texto,
        "accion": None,
        "raza": None,
        "cantidad": None,
        "origen": None,
        "destino": None,
        "completo": False,
        "ambiguo": []
    }

    # 1. EXTRAER ACCIÓN
    acciones_posibles = ["agrupar", "juntar", "reunir", "mover", "llevar",
                         "trasladar", "separar", "apartar", "traer"]

    for token in doc:
        if token.lemma_ in acciones_posibles or token.text in acciones_posibles:
            info["accion"] = token.lemma_
            break

    # Si no se detectó, buscar en entidades
    for ent in doc.ents:
        if ent.label_ == "ACCION":
            info["accion"] = ent.text
            break

    # 2. EXTRAER RAZA
    for ent in doc.ents:
        if ent.label_ == "RAZA":
            raza_normalizada = ent.text.capitalize()
            # Normalizar plural
            if raza_normalizada.endswith('s'):
                raza_normalizada = raza_normalizada[:-1]
            if raza_normalizada == "Merinas":
                raza_normalizada = "Merina"
            info["raza"] = raza_normalizada
            break

    # Si dice "todas" sin especificar raza
    if any(token.text in ["todas", "todos"] for token in doc):
        if not info["raza"]:
            info["raza"] = "todas"

    # 3. EXTRAER CANTIDAD
    for token in doc:
        if token.pos_ == "NUM":
            info["cantidad"] = int(token.text)
            break

    # Palabras que indican cantidad aproximada (para lógica difusa)
    palabras_cantidad = {
        "todas": "todas",
        "algunos": "algunos",
        "unas": "algunas",
        "pocas": "pocas",
        "bastantes": "bastantes",
        "varias": "varias"
    }

    for token in doc:
        if token.text in palabras_cantidad:
            if not info["cantidad"]:
                info["cantidad"] = palabras_cantidad[token.text]
            break

    # 4. EXTRAER UBICACIONES (ORIGEN y DESTINO)
    ubicaciones_detectadas = []
    for ent in doc.ents:
        if ent.label_ == "UBICACION":
            ubicaciones_detectadas.append(ent.text)

    # Identificar origen y destino según preposiciones
    for i, token in enumerate(doc):
        if token.text in ["de", "desde"] and i+1 < len(doc):
            # Lo que viene después es origen
            for ent in doc.ents:
                if ent.label_ == "UBICACION" and ent.start >= i:
                    info["origen"] = ent.text
                    break

        if token.text in ["a", "al", "hacia", "en"] and i+1 < len(doc):
            # Lo que viene después es destino
            for ent in doc.ents:
                if ent.label_ == "UBICACION" and ent.start >= i:
                    info["destino"] = ent.text
                    break

    # Si solo hay una ubicación y la acción implica movimiento, es destino
    if len(ubicaciones_detectadas) == 1 and not info["destino"]:
        if info["accion"] in ["mover", "llevar", "trasladar"]:
            info["destino"] = ubicaciones_detectadas[0]

    # 5. VERIFICAR SI LA INFORMACIÓN ESTÁ COMPLETA
    info["completo"] = all([info["accion"], info["raza"]])

    # 6. DETECTAR AMBIGÜEDADES
    if not info["accion"]:
        info["ambiguo"].append("accion")
    if not info["raza"]:
        info["ambiguo"].append("raza")
    if info["accion"] in ["mover", "llevar"] and not info["destino"]:
        info["ambiguo"].append("destino")

    return info

# Función para mostrar la extracción de forma visual
def mostrar_extraccion(info):
    """Muestra la información extraída de forma clara"""
    print("\n" + "="*70)
    print("🔍 ANÁLISIS DE LA ORDEN")
    print("="*70)
    print(f"📝 Texto: {info['texto_original']}")
    print(f"\n🎯 Extracción:")
    print(f"   • Acción: {info['accion'] or '❌ No detectada'}")
    print(f"   • Raza: {info['raza'] or '❌ No especificada'}")
    print(f"   • Cantidad: {info['cantidad'] or 'Todas'}")
    print(f"   • Origen: {info['origen'] or 'Ubicación actual'}")
    print(f"   • Destino: {info['destino'] or '❌ No especificado'}")

    if info["completo"]:
        print(f"\n✅ Información completa - Lista para ejecutar")
    else:
        print(f"\n⚠️ Información incompleta")
        print(f"   Falta: {', '.join(info['ambiguo'])}")
    print("="*70)

# Pruebas del sistema de extracción
print("✅ Sistema de extracción de entidades y relaciones configurado\n")

# Prueba 1
texto1 = "Agrupa todas las ovejas Merina en el corral norte"
info1 = extraer_informacion(texto1)
mostrar_extraccion(info1)

# Prueba 2
texto2 = "Lleva 10 Suffolk de la pradera este al cobertizo"
info2 = extraer_informacion(texto2)
mostrar_extraccion(info2)

# Prueba 3 (con ambigüedad)
texto3 = "Separa las Dorper"
info3 = extraer_informacion(texto3)
mostrar_extraccion(info3)


✅ Sistema de extracción de entidades y relaciones configurado


🔍 ANÁLISIS DE LA ORDEN
📝 Texto: Agrupa todas las ovejas Merina en el corral norte

🎯 Extracción:
   • Acción: agrupa
   • Raza: Merina
   • Cantidad: todas
   • Origen: Ubicación actual
   • Destino: corral norte

✅ Información completa - Lista para ejecutar

🔍 ANÁLISIS DE LA ORDEN
📝 Texto: Lleva 10 Suffolk de la pradera este al cobertizo

🎯 Extracción:
   • Acción: lleva
   • Raza: Suffolk
   • Cantidad: 10
   • Origen: pradera este
   • Destino: cobertizo

✅ Información completa - Lista para ejecutar

🔍 ANÁLISIS DE LA ORDEN
📝 Texto: Separa las Dorper

🎯 Extracción:
   • Acción: separa
   • Raza: Dorper
   • Cantidad: Todas
   • Origen: Ubicación actual
   • Destino: ❌ No especificado

✅ Información completa - Lista para ejecutar


In [10]:
# ============================================
# MANEJO DE AMBIGÜEDAD Y RAZONAMIENTO IMPRECISO
# ============================================

# Función para interpretar cantidades difusas
def interpretar_cantidad_difusa(cantidad_texto, raza, ubicacion=None):
    """
    Convierte expresiones difusas en números concretos
    Usa reglas difusas basadas en el total disponible
    """
    # Obtener total disponible
    if ubicacion:
        total = len(df_ovejas[(df_ovejas['raza'] == raza) &
                              (df_ovejas['ubicacion'] == ubicacion)])
    else:
        total = len(df_ovejas[df_ovejas['raza'] == raza])

    if total == 0:
        return 0

    # Reglas difusas
    reglas_difusas = {
        "pocas": 0.15,      # 15% del total
        "algunas": 0.3,     # 30% del total
        "varias": 0.5,      # 50% del total
        "bastantes": 0.7,   # 70% del total
        "muchas": 0.85,     # 85% del total
        "todas": 1.0        # 100%
    }

    if cantidad_texto in reglas_difusas:
        porcentaje = reglas_difusas[cantidad_texto]
        cantidad_real = max(1, int(total * porcentaje))
        return cantidad_real

    return total  # Por defecto, todas

# Función para solicitar información faltante
def solicitar_informacion_faltante(info):
    """
    Detecta qué información falta y la solicita al usuario
    Retorna el mensaje a enviar por voz/texto
    """
    mensajes = []

    if not info["accion"]:
        mensajes.append("No he entendido qué acción quieres realizar")
        mensajes.append("¿Quieres agrupar, mover o separar las ovejas?")

    if not info["raza"]:
        mensajes.append("No has especificado qué tipo de ovejas")
        mensajes.append("¿Te refieres a Merina, Manchega, Suffolk, Dorper o Corriedale?")

    if info["accion"] in ["mover", "llevar", "trasladar"] and not info["destino"]:
        mensajes.append("No has indicado hacia dónde moverlas")
        mensajes.append(f"Ubicaciones disponibles: {', '.join(UBICACIONES)}")

    if info["accion"] in ["separar", "apartar"] and not info["destino"]:
        mensajes.append("¿A dónde quieres separar las ovejas?")

    return " ".join(mensajes)

# Función para completar información con valores por defecto
def completar_con_defaults(info):
    """
    Completa información faltante con valores razonables por defecto
    """
    info_completada = info.copy()

    # Si no hay cantidad, asumir "todas"
    if not info_completada["cantidad"]:
        info_completada["cantidad"] = "todas"

    # Si no hay origen, buscar dónde están esas ovejas
    if not info_completada["origen"] and info_completada["raza"]:
        if info_completada["raza"] != "todas":
            ubicacion_actual = df_ovejas[df_ovejas['raza'] == info_completada['raza']]['ubicacion'].mode()
            if len(ubicacion_actual) > 0:
                info_completada["origen"] = ubicacion_actual[0]

    # Si la acción es "agrupar" y no hay destino, usar la ubicación más común
    if info_completada["accion"] in ["agrupar", "juntar"] and not info_completada["destino"]:
        info_completada["destino"] = "corral norte"  # Default para agrupaciones

    return info_completada

# Sistema de confirmación inteligente
def generar_confirmacion(info):
    """
    Genera un mensaje de confirmación natural antes de ejecutar
    """
    raza_texto = info['raza'] if info['raza'] != 'todas' else 'todas las ovejas'
    cantidad = info['cantidad'] if info['cantidad'] else 'todas las'

    if info['accion'] in ['mover', 'llevar', 'trasladar']:
        return f"Voy a {info['accion']} {cantidad} {raza_texto} hacia {info['destino']}"
    elif info['accion'] in ['agrupar', 'juntar']:
        return f"Voy a {info['accion']} las {raza_texto} en {info['destino']}"
    elif info['accion'] in ['separar', 'apartar']:
        return f"Voy a {info['accion']} {cantidad} {raza_texto}"
    else:
        return f"Voy a realizar la acción: {info['accion']}"

print("✅ Sistema de manejo de ambigüedad configurado")
print("\n🧠 Capacidades de razonamiento:")
print("   • Interpretación de cantidades difusas (pocas, bastantes, etc.)")
print("   • Detección de información faltante")
print("   • Completado automático con valores por defecto")
print("   • Generación de mensajes de confirmación")

# Prueba del sistema
print("\n" + "="*70)
print("🧪 PRUEBA DE RAZONAMIENTO IMPRECISO")
print("="*70)

# Caso 1: Cantidad difusa
print("\n📊 Caso 1: Interpretación de 'pocas ovejas Merina'")
cantidad_pocas = interpretar_cantidad_difusa("pocas", "Merina")
total_merinas = len(df_ovejas[df_ovejas['raza'] == "Merina"])
print(f"   Total Merinas disponibles: {total_merinas}")
print(f"   'Pocas' se interpreta como: {cantidad_pocas} ovejas (15%)")

# Caso 2: Información incompleta
print("\n⚠️ Caso 2: Orden incompleta")
info_incompleta = extraer_informacion("Mueve las Merina")
mensaje = solicitar_informacion_faltante(info_incompleta)
print(f"   Orden: 'Mueve las Merina'")
print(f"   Falta: {', '.join(info_incompleta['ambiguo'])}")
print(f"   Pregunta al usuario: {mensaje}")

# Caso 3: Completar con defaults
print("\n✨ Caso 3: Completar con valores por defecto")
info_completa = completar_con_defaults(info_incompleta)
confirmacion = generar_confirmacion(info_completa)
print(f"   Información completada automáticamente")
print(f"   Confirmación: '{confirmacion}'")


✅ Sistema de manejo de ambigüedad configurado

🧠 Capacidades de razonamiento:
   • Interpretación de cantidades difusas (pocas, bastantes, etc.)
   • Detección de información faltante
   • Completado automático con valores por defecto
   • Generación de mensajes de confirmación

🧪 PRUEBA DE RAZONAMIENTO IMPRECISO

📊 Caso 1: Interpretación de 'pocas ovejas Merina'
   Total Merinas disponibles: 98
   'Pocas' se interpreta como: 14 ovejas (15%)

⚠️ Caso 2: Orden incompleta
   Orden: 'Mueve las Merina'
   Falta: 
   Pregunta al usuario: 

✨ Caso 3: Completar con valores por defecto
   Información completada automáticamente
   Confirmación: 'Voy a realizar la acción: mueve'


In [12]:
# ============================================
# MOTOR DE EJECUCIÓN DE ACCIONES
# ============================================

import random

# Log de acciones realizadas (para trazabilidad)
log_acciones = []

def registrar_accion(accion_info, resultado):
    """Registra cada acción en el log para trazabilidad"""
    registro = {
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "accion": accion_info["accion"],
        "raza": accion_info["raza"],
        "cantidad": accion_info["cantidad"],
        "origen": accion_info["origen"],
        "destino": accion_info["destino"],
        "resultado": resultado,
        "ovejas_movidas": resultado.get("ovejas_movidas", 0)
    }
    log_acciones.append(registro)

def ejecutar_accion(info):
    """
    Ejecuta la acción sobre el dataframe de ovejas
    Actualiza el estado del rebaño
    """
    global df_ovejas

    # Completar información si falta algo
    info = completar_con_defaults(info)

    resultado = {
        "exito": False,
        "mensaje": "",
        "ovejas_movidas": 0,
        "interjeccion": ""
    }

    # Validar que tengamos lo mínimo
    if not info["accion"] or not info["raza"]:
        resultado["mensaje"] = "No puedo ejecutar la acción. Falta información crítica."
        return resultado

    # Obtener las ovejas afectadas
    if info["raza"] == "todas":
        if info["origen"]:
            ovejas_seleccionadas = df_ovejas[df_ovejas['ubicacion'] == info['origen']]
        else:
            ovejas_seleccionadas = df_ovejas.copy()
    else:
        if info["origen"]:
            ovejas_seleccionadas = df_ovejas[(df_ovejas['raza'] == info['raza']) &
                                              (df_ovejas['ubicacion'] == info['origen'])]
        else:
            ovejas_seleccionadas = df_ovejas[df_ovejas['raza'] == info['raza']]

    # Aplicar cantidad
    if info["cantidad"]:
        if isinstance(info["cantidad"], int):
            num_a_mover = min(info["cantidad"], len(ovejas_seleccionadas))
        elif isinstance(info["cantidad"], str):
            if info["cantidad"] == "todas":
                num_a_mover = len(ovejas_seleccionadas)
            else:
                # Cantidad difusa
                num_a_mover = interpretar_cantidad_difusa(info["cantidad"],
                                                          info["raza"],
                                                          info["origen"])
    else:
        num_a_mover = len(ovejas_seleccionadas)

    # Limitar al número disponible
    num_a_mover = min(num_a_mover, len(ovejas_seleccionadas))

    if num_a_mover == 0:
        resultado["mensaje"] = f"No hay ovejas {info['raza']} disponibles en {info['origen'] or 'ninguna ubicación'}."
        return resultado

    # Seleccionar ovejas al azar
    indices_a_mover = ovejas_seleccionadas.sample(n=num_a_mover).index

    # EJECUTAR LA ACCIÓN
    # Normalizar acción
    accion_base = info["accion"]

    if info["accion"] in ["mover", "mueve", "lleva", "llevar", "traslada", "trasladar",
                          "agrupar", "agrupa", "juntar", "junta", "reunir"]:
        if not info["destino"]:
            resultado["mensaje"] = "No se puede mover sin destino."
            return resultado

        # Actualizar ubicación
        df_ovejas.loc[indices_a_mover, 'ubicacion'] = info["destino"]

        resultado["exito"] = True
        resultado["ovejas_movidas"] = num_a_mover
        resultado["mensaje"] = f"Se han movido {num_a_mover} ovejas {info['raza']} a {info['destino']}"
        resultado["interjeccion"] = obtener_interjeccion("mover", info["destino"])

    elif info["accion"] in ["separar", "separa", "apartar", "aparta"]:
        # Separar significa mover a una ubicación diferente
        if not info["destino"]:
            info["destino"] = "cobertizo"  # Default para separaciones

        df_ovejas.loc[indices_a_mover, 'ubicacion'] = info["destino"]

        resultado["exito"] = True
        resultado["ovejas_movidas"] = num_a_mover
        resultado["mensaje"] = f"Se han separado {num_a_mover} ovejas {info['raza']} hacia {info['destino']}"
        resultado["interjeccion"] = obtener_interjeccion("separar")

    else:
        resultado["mensaje"] = f"Acción '{info['accion']}' no reconocida."
        return resultado

    # Registrar en el log
    registrar_accion(info, resultado)

    return resultado


# Función para mostrar el resultado de la ejecución
def mostrar_resultado(resultado):
    """Muestra el resultado de la acción ejecutada"""
    print("\n" + "="*70)
    print("⚙️ RESULTADO DE LA EJECUCIÓN")
    print("="*70)

    if resultado["exito"]:
        print(f"✅ {resultado['mensaje']}")
        print(f"\n🔊 Interjección de pastor: {resultado['interjeccion']}")
    else:
        print(f"❌ {resultado['mensaje']}")

    print("="*70)

print("✅ Motor de ejecución de acciones configurado")
print("📋 Log de acciones inicializado")

# Prueba completa del sistema
print("\n" + "="*70)
print("🧪 PRUEBA COMPLETA: ORDEN → EXTRACCIÓN → EJECUCIÓN")
print("="*70)

orden_prueba = "Lleva 15 ovejas Merina al corral norte"
print(f"\n📝 Orden: '{orden_prueba}'")

# Paso 1: Extraer
info = extraer_informacion(orden_prueba)
mostrar_extraccion(info)

# Paso 2: Ejecutar
resultado = ejecutar_accion(info)
mostrar_resultado(resultado)

# Paso 3: Mostrar estado actualizado
mostrar_estado()


✅ Motor de ejecución de acciones configurado
📋 Log de acciones inicializado

🧪 PRUEBA COMPLETA: ORDEN → EXTRACCIÓN → EJECUCIÓN

📝 Orden: 'Lleva 15 ovejas Merina al corral norte'

🔍 ANÁLISIS DE LA ORDEN
📝 Texto: Lleva 15 ovejas Merina al corral norte

🎯 Extracción:
   • Acción: lleva
   • Raza: Merina
   • Cantidad: 15
   • Origen: Ubicación actual
   • Destino: corral norte

✅ Información completa - Lista para ejecutar

⚙️ RESULTADO DE LA EJECUCIÓN
✅ Se han movido 15 ovejas Merina a corral norte

🔊 Interjección de pastor: ¡Vamos pa'llá! ¡Hacia corral norte!

🐑 ESTADO ACTUAL DEL REBAÑO

📍 CORRAL NORTE: 15 ovejas
   • Merina: 15

📍 PRADERA ESTE: 485 ovejas
   • Merina: 83
   • Manchega: 117
   • Suffolk: 92
   • Dorper: 73
   • Corriedale: 120

🐏 TOTAL REBAÑO: 500 ovejas


In [13]:
# ============================================
# SISTEMA TTS - TEXT TO SPEECH
# ============================================

from gtts import gTTS
from IPython.display import Audio, display
import io

def generar_audio(texto, reproducir=True):
    """
    Genera audio a partir de texto usando gTTS
    Retorna el objeto Audio para reproducción en Colab
    """
    try:
        # Generar audio con gTTS (español)
        tts = gTTS(text=texto, lang='es', slow=False)

        # Guardar en memoria
        audio_buffer = io.BytesIO()
        tts.write_to_fp(audio_buffer)
        audio_buffer.seek(0)

        # Crear objeto Audio
        audio = Audio(audio_buffer.read(), autoplay=reproducir)

        return audio

    except Exception as e:
        print(f"❌ Error al generar audio: {e}")
        return None

def hablar(texto, mostrar_texto=True):
    """
    Función principal para que el sistema 'hable'
    Muestra el texto y reproduce el audio
    """
    if mostrar_texto:
        print(f"\n🔊 Sistema dice: '{texto}'")

    audio = generar_audio(texto, reproducir=True)
    if audio:
        display(audio)

    return audio

# Función para reproducir interjección de pastor
def reproducir_interjeccion(interjeccion):
    """
    Reproduce una interjección de pastor con énfasis
    """
    print(f"\n🐑 Interjección de pastor: {interjeccion}")
    audio = generar_audio(interjeccion, reproducir=True)
    if audio:
        display(audio)
    return audio

print("✅ Sistema TTS (Text-to-Speech) configurado")
print("🔊 El sistema puede generar y reproducir audio en español")

# Pruebas del sistema TTS
print("\n" + "="*70)
print("🧪 PRUEBAS DEL SISTEMA TTS")
print("="*70)

print("\n1️⃣ Mensaje simple:")
hablar("Sistema de pastoreo inteligente activado")

print("\n2️⃣ Interjección de pastor:")
reproducir_interjeccion("¡Arre, arre! ¡Vamos pa'l corral!")

print("\n3️⃣ Confirmación de acción:")
hablar("Voy a mover quince ovejas Merina al corral norte")


✅ Sistema TTS (Text-to-Speech) configurado
🔊 El sistema puede generar y reproducir audio en español

🧪 PRUEBAS DEL SISTEMA TTS

1️⃣ Mensaje simple:

🔊 Sistema dice: 'Sistema de pastoreo inteligente activado'



2️⃣ Interjección de pastor:

🐑 Interjección de pastor: ¡Arre, arre! ¡Vamos pa'l corral!



3️⃣ Confirmación de acción:

🔊 Sistema dice: 'Voy a mover quince ovejas Merina al corral norte'


In [19]:
# ============================================
# GENERACIÓN DE AUDIOS DE PRUEBA SINTÉTICOS
# ============================================

from gtts import gTTS
import os

def generar_audios_prueba():
    """
    Genera archivos de audio de prueba para testear el sistema ASR
    Sin necesidad de grabar con micrófono
    """
    print("="*70)
    print("🎵 GENERANDO AUDIOS DE PRUEBA SINTÉTICOS")
    print("="*70)

    # Lista de órdenes de prueba
    ordenes_prueba = [
        "Mueve todas las ovejas Merina al corral norte",
        "Lleva veinte Suffolk de la pradera este al cobertizo",
        "Agrupa las Corriedale en la nave",
        "Separa quince Dorper al ordeño",
        "Traslada treinta Manchega al corral sur",
        "Junta pocas Merina en la pradera oeste"
    ]

    archivos_generados = []

    for i, orden in enumerate(ordenes_prueba, 1):
        # Generar nombre de archivo
        filename = f"orden_prueba_{i}.mp3"

        # Crear audio con gTTS
        tts = gTTS(text=orden, lang='es', slow=False)
        tts.save(filename)

        archivos_generados.append(filename)
        print(f"✅ Generado: {filename}")
        print(f"   📝 Contenido: '{orden}'")

    print("\n" + "="*70)
    print(f"🎉 Se han generado {len(archivos_generados)} archivos de audio")
    print("="*70)
    print("\n📂 Archivos disponibles:")
    for archivo in archivos_generados:
        print(f"   • {archivo}")

    print("\n💡 Puedes usar estos archivos para probar el sistema ASR")
    print("   Ejemplo: procesar_comando_voz('orden_prueba_1.mp3')")

    return archivos_generados

# Generar los audios
archivos_audio = generar_audios_prueba()

print("\n" + "="*70)
print("🧪 VERIFICACIÓN DE ARCHIVOS")
print("="*70)

# Verificar que se crearon
for archivo in archivos_audio:
    if os.path.exists(archivo):
        tamaño = os.path.getsize(archivo)
        print(f"✅ {archivo} - {tamaño} bytes")
    else:
        print(f"❌ {archivo} - NO ENCONTRADO")

print("\n" + "="*70)
print("🎵 REPRODUCIR EJEMPLO")
print("="*70)
print("Escucha cómo suena el primer audio generado:")

# Reproducir el primer audio como ejemplo
if archivos_audio:
    display(Audio(archivos_audio[0], autoplay=False))
    print(f"\n🔊 Audio: {archivos_audio[0]}")


🎵 GENERANDO AUDIOS DE PRUEBA SINTÉTICOS
✅ Generado: orden_prueba_1.mp3
   📝 Contenido: 'Mueve todas las ovejas Merina al corral norte'
✅ Generado: orden_prueba_2.mp3
   📝 Contenido: 'Lleva veinte Suffolk de la pradera este al cobertizo'
✅ Generado: orden_prueba_3.mp3
   📝 Contenido: 'Agrupa las Corriedale en la nave'
✅ Generado: orden_prueba_4.mp3
   📝 Contenido: 'Separa quince Dorper al ordeño'
✅ Generado: orden_prueba_5.mp3
   📝 Contenido: 'Traslada treinta Manchega al corral sur'
✅ Generado: orden_prueba_6.mp3
   📝 Contenido: 'Junta pocas Merina en la pradera oeste'

🎉 Se han generado 6 archivos de audio

📂 Archivos disponibles:
   • orden_prueba_1.mp3
   • orden_prueba_2.mp3
   • orden_prueba_3.mp3
   • orden_prueba_4.mp3
   • orden_prueba_5.mp3
   • orden_prueba_6.mp3

💡 Puedes usar estos archivos para probar el sistema ASR
   Ejemplo: procesar_comando_voz('orden_prueba_1.mp3')

🧪 VERIFICACIÓN DE ARCHIVOS
✅ orden_prueba_1.mp3 - 28224 bytes
✅ orden_prueba_2.mp3 - 32256 bytes
✅ orde


🔊 Audio: orden_prueba_1.mp3


In [21]:
# ============================================
# SISTEMA ASR - RECONOCIMIENTO DE VOZ
# ============================================

import speech_recognition as sr
from pydub import AudioSegment
import os

# Inicializar reconocedor
recognizer = sr.Recognizer()

def transcribir_audio(ruta_archivo):
    """
    Transcribe un archivo de audio a texto
    Soporta formatos: WAV, MP3, OGG, FLAC
    """
    try:
        # Si el archivo no es WAV, convertir
        if not ruta_archivo.endswith('.wav'):
            print(f"🔄 Convirtiendo audio a formato WAV...")
            audio = AudioSegment.from_file(ruta_archivo)
            ruta_wav = ruta_archivo.rsplit('.', 1)[0] + '_converted.wav'
            audio.export(ruta_wav, format='wav')
            ruta_archivo = ruta_wav

        # Cargar audio
        with sr.AudioFile(ruta_archivo) as source:
            print("🎤 Procesando audio...")
            # Ajustar ruido ambiente
            recognizer.adjust_for_ambient_noise(source, duration=0.5)
            # Grabar audio
            audio_data = recognizer.record(source)

        # Transcribir con Google Speech Recognition (español)
        print("🔍 Transcribiendo...")
        texto = recognizer.recognize_google(audio_data, language='es-ES')

        print(f"✅ Transcripción exitosa")
        return texto

    except sr.UnknownValueError:
        print("❌ No se pudo entender el audio")
        return None
    except sr.RequestError as e:
        print(f"❌ Error en el servicio de reconocimiento: {e}")
        return None
    except Exception as e:
        print(f"❌ Error al procesar audio: {e}")
        return None

def procesar_comando_voz(ruta_archivo):
    """
    Procesa un comando de voz completo:
    1. Transcribe el audio
    2. Extrae información
    3. Ejecuta la acción
    4. Responde por voz
    """
    print("\n" + "="*70)
    print("🎤 PROCESANDO COMANDO DE VOZ")
    print("="*70)

    # Transcribir
    texto = transcribir_audio(ruta_archivo)

    if not texto:
        hablar("No he podido entender tu orden. Por favor, repite más claro.")
        return None

    print(f"\n📝 Texto transcrito: '{texto}'")

    # Extraer información
    info = extraer_informacion(texto)
    mostrar_extraccion(info)

    # Verificar si está completo
    if not info["completo"] or info["ambiguo"]:
        mensaje = solicitar_informacion_faltante(info)
        hablar(mensaje)
        return info

    # Confirmar
    confirmacion = generar_confirmacion(info)
    hablar(confirmacion)

    # Ejecutar
    resultado = ejecutar_accion(info)

    # Reproducir interjección
    if resultado["exito"]:
        reproducir_interjeccion(resultado["interjeccion"])
        hablar(resultado["mensaje"])
    else:
        hablar(resultado["mensaje"])

    # Mostrar estado
    mostrar_estado()

    return resultado

print("✅ Sistema ASR (Reconocimiento de Voz) configurado")
print("🎤 Soporta archivos: WAV, MP3, OGG, FLAC")
print("\n📝 Nota: Para probar, necesitas subir un archivo de audio con una orden")
print("   Ejemplo: 'Mueve las ovejas Merina al corral norte'")


✅ Sistema ASR (Reconocimiento de Voz) configurado
🎤 Soporta archivos: WAV, MP3, OGG, FLAC

📝 Nota: Para probar, necesitas subir un archivo de audio con una orden
   Ejemplo: 'Mueve las ovejas Merina al corral norte'


In [22]:
# ============================================
# MEJORA: CORRECCIÓN DE ERRORES COMUNES DE ASR
# ============================================

# Diccionario de correcciones comunes del reconocimiento de voz
CORRECCIONES_ASR = {
    "dime": "mueve",
    "dile": "lleva",
    "vine": "mueve",
    "leve": "lleva",
    "vente": "veinte",
    "kínce": "quince",
    "treintena": "treinta"
}

def corregir_errores_asr(texto):
    """
    Corrige errores comunes del reconocimiento de voz
    antes de procesar la orden
    """
    texto_original = texto
    texto_corregido = texto.lower()

    # Aplicar correcciones palabra por palabra
    palabras = texto_corregido.split()
    palabras_corregidas = []

    for palabra in palabras:
        if palabra in CORRECCIONES_ASR:
            palabras_corregidas.append(CORRECCIONES_ASR[palabra])
        else:
            palabras_corregidas.append(palabra)

    texto_corregido = " ".join(palabras_corregidas)

    if texto_corregido != texto_original.lower():
        print(f"🔧 Corrección ASR aplicada:")
        print(f"   Antes:  '{texto_original}'")
        print(f"   Después: '{texto_corregido}'")

    return texto_corregido

# Actualizar la función procesar_comando_voz para usar correcciones
def procesar_comando_voz_mejorado(ruta_archivo):
    """
    Versión mejorada con corrección de errores ASR
    """
    print("\n" + "="*70)
    print("🎤 PROCESANDO COMANDO DE VOZ (CON CORRECCIÓN ASR)")
    print("="*70)

    # Transcribir
    texto = transcribir_audio(ruta_archivo)

    if not texto:
        hablar("No he podido entender tu orden. Por favor, repite más claro.")
        return None

    print(f"\n📝 Texto transcrito: '{texto}'")

    # APLICAR CORRECCIONES
    texto = corregir_errores_asr(texto)

    # Extraer información
    info = extraer_informacion(texto)
    mostrar_extraccion(info)

    # Verificar si está completo
    if not info["completo"] or info["ambiguo"]:
        mensaje = solicitar_informacion_faltante(info)
        hablar(mensaje)
        return info

    # Confirmar
    confirmacion = generar_confirmacion(info)
    hablar(confirmacion)

    # Ejecutar
    resultado = ejecutar_accion(info)

    # Reproducir interjección
    if resultado["exito"]:
        reproducir_interjeccion(resultado["interjeccion"])
        hablar(resultado["mensaje"])
    else:
        hablar(resultado["mensaje"])

    # Mostrar estado
    mostrar_estado()

    return resultado

print("✅ Sistema de corrección de errores ASR configurado")
print(f"📝 Correcciones disponibles: {len(CORRECCIONES_ASR)}")

# Probar de nuevo con corrección
print("\n" + "="*70)
print("🧪 REPRUEBA CON CORRECCIÓN AUTOMÁTICA")
print("="*70)

resultado = procesar_comando_voz_mejorado(archivos_audio[0])


✅ Sistema de corrección de errores ASR configurado
📝 Correcciones disponibles: 7

🧪 REPRUEBA CON CORRECCIÓN AUTOMÁTICA

🎤 PROCESANDO COMANDO DE VOZ (CON CORRECCIÓN ASR)
🔄 Convirtiendo audio a formato WAV...
🎤 Procesando audio...
🔍 Transcribiendo...
✅ Transcripción exitosa

📝 Texto transcrito: 'dime todas las ovejas merina al corral norte'
🔧 Corrección ASR aplicada:
   Antes:  'dime todas las ovejas merina al corral norte'
   Después: 'mueve todas las ovejas merina al corral norte'

🔍 ANÁLISIS DE LA ORDEN
📝 Texto: mueve todas las ovejas merina al corral norte

🎯 Extracción:
   • Acción: mueve
   • Raza: Merina
   • Cantidad: todas
   • Origen: Ubicación actual
   • Destino: corral norte

✅ Información completa - Lista para ejecutar

🔊 Sistema dice: 'Voy a realizar la acción: mueve'



🐑 Interjección de pastor: ¡Arre hacia allá! ¡Hacia corral norte!



🔊 Sistema dice: 'Se han movido 83 ovejas Merina a corral norte'



🐑 ESTADO ACTUAL DEL REBAÑO

📍 CORRAL NORTE: 98 ovejas
   • Merina: 98

📍 PRADERA ESTE: 260 ovejas
   • Manchega: 87
   • Dorper: 73
   • Corriedale: 100

📍 NAVE: 92 ovejas
   • Suffolk: 92

📍 COBERTIZO: 20 ovejas
   • Corriedale: 20

📍 ORDEÑO: 30 ovejas
   • Manchega: 30

🐏 TOTAL REBAÑO: 500 ovejas


In [23]:
# ============================================
# SISTEMA INTEGRADO COMPLETO
# ============================================

def procesar_orden_completa(texto_orden=None, archivo_audio=None, modo="texto"):
    """
    Función principal que procesa una orden completa
    Puede recibir texto directo o archivo de audio

    Parámetros:
    - texto_orden: string con la orden (si modo="texto")
    - archivo_audio: ruta al archivo de audio (si modo="voz")
    - modo: "texto" o "voz"
    """
    print("\n" + "="*70)
    print("🐑 SIMULADOR DE PASTOREO INTELIGENTE")
    print("="*70)

    # PASO 1: Obtener el texto de la orden
    if modo == "voz":
        if not archivo_audio:
            hablar("Error: No se proporcionó archivo de audio")
            return None

        print("\n🎤 Modo: ENTRADA POR VOZ")
        print("="*70)

        # Usar la función mejorada con corrección ASR
        return procesar_comando_voz_mejorado(archivo_audio)

    else:
        print("\n⌨️ Modo: ENTRADA POR TEXTO")
        print("="*70)
        print(f"📝 Orden recibida: '{texto_orden}'")

    # PASO 2: Extraer información
    print("\n" + "="*70)
    print("🔍 FASE 1: ANÁLISIS DE LA ORDEN")
    print("="*70)

    info = extraer_informacion(texto_orden)
    mostrar_extraccion(info)

    # PASO 3: Verificar completitud y manejar ambigüedad
    if not info["completo"] or info["ambiguo"]:
        print("\n⚠️ La orden está incompleta")
        mensaje_faltante = solicitar_informacion_faltante(info)
        print(f"\n❓ Pregunta al usuario: {mensaje_faltante}")
        hablar(mensaje_faltante)
        return {"info": info, "estado": "incompleto"}

    # PASO 4: Confirmar acción
    print("\n" + "="*70)
    print("✅ FASE 2: CONFIRMACIÓN")
    print("="*70)

    confirmacion = generar_confirmacion(info)
    print(f"📢 {confirmacion}")
    hablar(confirmacion)

    # PASO 5: Ejecutar acción
    print("\n" + "="*70)
    print("⚙️ FASE 3: EJECUCIÓN")
    print("="*70)

    resultado = ejecutar_accion(info)

    # PASO 6: Responder con interjección de pastor
    if resultado["exito"]:
        print(f"\n✅ {resultado['mensaje']}")
        print(f"🔊 Interjección: {resultado['interjeccion']}")

        # Reproducir interjección
        reproducir_interjeccion(resultado['interjeccion'])

        # Confirmar resultado
        hablar(resultado['mensaje'])
    else:
        print(f"\n❌ {resultado['mensaje']}")
        hablar(resultado['mensaje'])

    # PASO 7: Mostrar estado actualizado
    print("\n" + "="*70)
    print("📊 FASE 4: ESTADO ACTUALIZADO DEL REBAÑO")
    print("="*70)
    mostrar_estado()

    return {"info": info, "resultado": resultado, "estado": "completado"}

print("✅ Sistema integrado completo configurado")
print("\n🎯 Uso:")
print("   • Por texto: procesar_orden_completa('Mueve las Merina al corral', modo='texto')")
print("   • Por voz:   procesar_orden_completa(archivo_audio='orden.wav', modo='voz')")


✅ Sistema integrado completo configurado

🎯 Uso:
   • Por texto: procesar_orden_completa('Mueve las Merina al corral', modo='texto')
   • Por voz:   procesar_orden_completa(archivo_audio='orden.wav', modo='voz')


In [24]:
# ============================================
# INTERFAZ SIMPLIFICADA PARA COMANDOS DE TEXTO
# ============================================

def ordenar(texto):
    """
    Función simplificada para dar órdenes rápidas por texto
    Uso: ordenar("Mueve las Merina al corral norte")
    """
    return procesar_orden_completa(texto, modo="texto")

def ordenar_voz(archivo):
    """
    Función simplificada para procesar audio
    Uso: ordenar_voz("mi_orden.wav")
    """
    return procesar_orden_completa(archivo_audio=archivo, modo="voz")

print("✅ Funciones simplificadas configuradas")
print("\n🚀 EJEMPLOS DE USO:")
print("="*70)

# Ejemplo 1: Orden por texto
print("\n📝 Ejemplo 1: Orden por texto")
print("Código: ordenar('Agrupa todas las Suffolk en la nave')")
print("-" * 70)


✅ Funciones simplificadas configuradas

🚀 EJEMPLOS DE USO:

📝 Ejemplo 1: Orden por texto
Código: ordenar('Agrupa todas las Suffolk en la nave')
----------------------------------------------------------------------


In [25]:
# ============================================
# GENERACIÓN DE DATASET DE ÓRDENES DE PRUEBA
# ============================================

import pandas as pd
import random

# Generar ejemplos variados de órdenes
def generar_dataset_ordenes(num_ejemplos=50):
    """
    Genera un dataset de órdenes de prueba para entrenar y validar
    """
    razas = ["Merina", "Manchega", "Suffolk", "Dorper", "Corriedale"]
    acciones = ["Mueve", "Lleva", "Agrupa", "Junta", "Separa", "Traslada"]
    ubicaciones = UBICACIONES

    ordenes = []

    for i in range(num_ejemplos):
        tipo = random.choice(["simple", "completa", "ambigua", "difusa"])

        if tipo == "simple":
            # Orden básica: acción + raza + destino
            accion = random.choice(acciones)
            raza = random.choice(razas)
            destino = random.choice(ubicaciones)
            orden = f"{accion} las ovejas {raza} al {destino}"

        elif tipo == "completa":
            # Orden completa: acción + cantidad + raza + origen + destino
            accion = random.choice(["Mueve", "Lleva", "Traslada"])
            cantidad = random.randint(5, 30)
            raza = random.choice(razas)
            origen = random.choice(ubicaciones)
            destino = random.choice([u for u in ubicaciones if u != origen])
            orden = f"{accion} {cantidad} ovejas {raza} de {origen} a {destino}"

        elif tipo == "ambigua":
            # Orden incompleta
            accion = random.choice(acciones)
            raza = random.choice(razas)
            orden = f"{accion} las {raza}"

        else:  # difusa
            # Orden con cantidad difusa
            accion = random.choice(acciones)
            cantidad_difusa = random.choice(["pocas", "algunas", "varias", "bastantes", "muchas"])
            raza = random.choice(razas)
            destino = random.choice(ubicaciones)
            orden = f"{accion} {cantidad_difusa} {raza} al {destino}"

        ordenes.append({
            "id": i+1,
            "orden": orden,
            "tipo": tipo,
            "raza_esperada": raza if tipo != "simple" or random.random() > 0.2 else None
        })

    return pd.DataFrame(ordenes)

# Generar dataset
df_ordenes = generar_dataset_ordenes(50)

print("✅ Dataset de órdenes de prueba generado")
print(f"📊 Total de ejemplos: {len(df_ordenes)}")
print(f"\n📋 Distribución por tipo:")
print(df_ordenes['tipo'].value_counts())

print("\n" + "="*70)
print("🔍 EJEMPLOS DEL DATASET")
print("="*70)
display(df_ordenes.head(15))

# Guardar dataset
df_ordenes.to_csv('dataset_ordenes_prueba.csv', index=False)
print("\n💾 Dataset guardado como 'dataset_ordenes_prueba.csv'")

# Probar algunas órdenes del dataset
print("\n" + "="*70)
print("🧪 PRUEBA DE ÓRDENES DEL DATASET")
print("="*70)

for i in [0, 5, 10]:
    orden = df_ordenes.iloc[i]['orden']
    print(f"\n{'='*70}")
    print(f"Probando orden #{i+1}: '{orden}'")
    print(f"{'='*70}")
    info = extraer_informacion(orden)
    mostrar_extraccion(info)


✅ Dataset de órdenes de prueba generado
📊 Total de ejemplos: 50

📋 Distribución por tipo:
tipo
completa    16
simple      12
difusa      12
ambigua     10
Name: count, dtype: int64

🔍 EJEMPLOS DEL DATASET


,id,orden,tipo,raza_esperada
0,1,Traslada 23 ovejas Manchega de pradera oeste a...,completa,Manchega
1,2,Traslada 25 ovejas Merina de pradera este a co...,completa,Merina
2,3,Traslada las ovejas Dorper al cobertizo,simple,Dorper
3,4,Traslada 22 ovejas Corriedale de pradera oeste...,completa,Corriedale
4,5,Mueve las Manchega,ambigua,Manchega
5,6,Lleva bastantes Dorper al pradera este,difusa,Dorper
6,7,Agrupa algunas Dorper al nave,difusa,Dorper
7,8,Mueve 18 ovejas Suffolk de cobertizo a ordeño,completa,Suffolk
8,9,Mueve 10 ovejas Corriedale de nave a corral norte,completa,Corriedale
9,10,Lleva las Corriedale,ambigua,Corriedale



💾 Dataset guardado como 'dataset_ordenes_prueba.csv'

🧪 PRUEBA DE ÓRDENES DEL DATASET

Probando orden #1: 'Traslada 23 ovejas Manchega de pradera oeste a ordeño'

🔍 ANÁLISIS DE LA ORDEN
📝 Texto: Traslada 23 ovejas Manchega de pradera oeste a ordeño

🎯 Extracción:
   • Acción: traslada
   • Raza: Manchega
   • Cantidad: 23
   • Origen: pradera oeste
   • Destino: ordeño

✅ Información completa - Lista para ejecutar

Probando orden #6: 'Lleva bastantes Dorper al pradera este'

🔍 ANÁLISIS DE LA ORDEN
📝 Texto: Lleva bastantes Dorper al pradera este

🎯 Extracción:
   • Acción: lleva
   • Raza: Dorper
   • Cantidad: bastantes
   • Origen: Ubicación actual
   • Destino: pradera este

✅ Información completa - Lista para ejecutar

Probando orden #11: 'Agrupa las ovejas Corriedale al corral sur'

🔍 ANÁLISIS DE LA ORDEN
📝 Texto: Agrupa las ovejas Corriedale al corral sur

🎯 Extracción:
   • Acción: agrupa
   • Raza: Corriedale
   • Cantidad: Todas
   • Origen: Ubicación actual
   • Destino: corr

In [26]:
# ============================================
# VISUALIZACIÓN Y ANÁLISIS DEL LOG DE ACCIONES
# ============================================

def mostrar_log_acciones():
    """
    Muestra el registro completo de acciones realizadas en el simulador
    """
    if not log_acciones:
        print("📝 No hay acciones registradas aún.")
        return

    print("\n" + "="*70)
    print("📜 REGISTRO DE ACCIONES (LOG DE TRAZABILIDAD)")
    print("="*70)

    df_log = pd.DataFrame(log_acciones)

    print(f"\n📊 Total de acciones ejecutadas: {len(df_log)}")
    print(f"🐑 Total de ovejas movidas: {df_log['ovejas_movidas'].sum()}")

    print("\n📋 Últimas 10 acciones realizadas:")
    display(df_log[['timestamp', 'accion', 'raza', 'cantidad', 'origen', 'destino', 'ovejas_movidas']].tail(10))

    # Estadísticas
    print("\n" + "="*70)
    print("📈 ESTADÍSTICAS DE USO")
    print("="*70)
    print(f"\n🎯 Acciones más frecuentes:")
    print(df_log['accion'].value_counts())

    print(f"\n🐏 Razas más movidas:")
    print(df_log['raza'].value_counts())

    print(f"\n📍 Destinos más usados:")
    print(df_log['destino'].value_counts())

    return df_log

def exportar_log():
    """Exporta el log de acciones a un archivo CSV"""
    if log_acciones:
        df_log = pd.DataFrame(log_acciones)
        df_log.to_csv('log_acciones_pastoreo.csv', index=False)
        print("💾 Log exportado como 'log_acciones_pastoreo.csv'")
    else:
        print("⚠️ No hay acciones que exportar.")

def limpiar_log():
    """Limpia el log de acciones realizadas"""
    global log_acciones
    log_acciones = []
    print("🗑️ Log de acciones limpiado.")

print("✅ Sistema de log y trazabilidad configurado")
print("\n📊 Funciones disponibles:")
print("   • mostrar_log_acciones() - Ver todas las acciones")
print("   • exportar_log() - Guardar log en CSV")
print("   • limpiar_log() - Limpiar el log")

# Mostrar log actual
mostrar_log_acciones()


✅ Sistema de log y trazabilidad configurado

📊 Funciones disponibles:
   • mostrar_log_acciones() - Ver todas las acciones
   • exportar_log() - Guardar log en CSV
   • limpiar_log() - Limpiar el log

📜 REGISTRO DE ACCIONES (LOG DE TRAZABILIDAD)

📊 Total de acciones ejecutadas: 5
🐑 Total de ovejas movidas: 240

📋 Últimas 10 acciones realizadas:


,timestamp,accion,raza,cantidad,origen,destino,ovejas_movidas
0,2025-11-19 16:44:20,lleva,Merina,15,pradera este,corral norte,15
1,2025-11-19 16:47:28,agrupa,Suffolk,todas,pradera este,nave,92
2,2025-11-19 16:47:28,separa,Corriedale,20,pradera este,cobertizo,20
3,2025-11-19 16:47:28,lleva,Manchega,30,pradera este,ordeño,30
4,2025-11-19 16:52:49,mueve,Merina,todas,pradera este,corral norte,83



📈 ESTADÍSTICAS DE USO

🎯 Acciones más frecuentes:
accion
lleva     2
agrupa    1
separa    1
mueve     1
Name: count, dtype: int64

🐏 Razas más movidas:
raza
Merina        2
Suffolk       1
Corriedale    1
Manchega      1
Name: count, dtype: int64

📍 Destinos más usados:
destino
corral norte    2
nave            1
cobertizo       1
ordeño          1
Name: count, dtype: int64


,timestamp,accion,raza,cantidad,origen,destino,resultado,ovejas_movidas
0,2025-11-19 16:44:20,lleva,Merina,15,pradera este,corral norte,"{'exito': True, 'mensaje': 'Se han movido 15 o...",15
1,2025-11-19 16:47:28,agrupa,Suffolk,todas,pradera este,nave,"{'exito': True, 'mensaje': 'Se han movido 92 o...",92
2,2025-11-19 16:47:28,separa,Corriedale,20,pradera este,cobertizo,"{'exito': True, 'mensaje': 'Se han separado 20...",20
3,2025-11-19 16:47:28,lleva,Manchega,30,pradera este,ordeño,"{'exito': True, 'mensaje': 'Se han movido 30 o...",30
4,2025-11-19 16:52:49,mueve,Merina,todas,pradera este,corral norte,"{'exito': True, 'mensaje': 'Se han movido 83 o...",83


In [27]:
# ============================================
# INTERFAZ DE USUARIO FINAL Y MENÚ DE OPCIONES
# ============================================

from IPython.display import display, HTML

def mostrar_menu():
    """Muestra el menú principal del simulador"""
    menu_html = """
    <div style="border: 3px solid #2196F3; border-radius: 15px; padding: 30px;
                background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                color: white; max-width: 800px; margin: 20px auto; box-shadow: 0 10px 30px rgba(0,0,0,0.3);">

        <h1 style="text-align: center; font-size: 32px; margin-bottom: 20px;">
            🐑 SIMULADOR INTELIGENTE DE PASTOREO 🐑
        </h1>

        <div style="background-color: rgba(255,255,255,0.1); padding: 20px; border-radius: 10px; margin: 20px 0;">
            <h2 style="margin-top: 0;">📋 Opciones disponibles:</h2>
            <ul style="font-size: 16px; line-height: 1.8;">
                <li><strong>Por Texto:</strong> <code>ordenar("tu orden aquí")</code></li>
                <li><strong>Por Voz:</strong>
                    <ol>
                        <li><code>mostrar_grabador()</code> - Grabar orden</li>
                        <li><code>archivo = subir_audio()</code> - Subir archivo</li>
                        <li><code>ordenar_voz(archivo)</code> - Procesar audio</li>
                    </ol>
                </li>
                <li><strong>Ver Estado:</strong> <code>mostrar_estado()</code></li>
                <li><strong>Ver Log:</strong> <code>mostrar_log_acciones()</code></li>
                <li><strong>Exportar Datos:</strong> <code>exportar_log()</code></li>
            </ul>
        </div>

        <div style="background-color: rgba(255,255,255,0.1); padding: 20px; border-radius: 10px; margin: 20px 0;">
            <h2 style="margin-top: 0;">🎯 Ejemplos de órdenes:</h2>
            <ul style="font-size: 14px; line-height: 1.6;">
                <li>"Mueve todas las ovejas Merina al corral norte"</li>
                <li>"Lleva 20 Suffolk de la pradera este al cobertizo"</li>
                <li>"Agrupa las Corriedale en la nave"</li>
                <li>"Separa pocas Dorper al ordeño"</li>
                <li>"Traslada 15 Manchega al corral sur"</li>
            </ul>
        </div>

        <div style="background-color: rgba(255,255,255,0.2); padding: 15px; border-radius: 10px; text-align: center;">
            <p style="margin: 0; font-size: 18px; font-weight: bold;">
                🚀 Sistema listo para usar
            </p>
        </div>
    </div>
    """
    display(HTML(menu_html))

mostrar_menu()

print("\n" + "="*70)
print("✅ SISTEMA COMPLETAMENTE OPERATIVO")
print("="*70)
print(f"🐑 Rebaño inicial: 500 ovejas")
print(f"📍 Ubicaciones: {len(UBICACIONES)}")
print(f"🏷️ Razas: 5 (Merina, Manchega, Suffolk, Dorper, Corriedale)")
print(f"🎤 Modos: Texto y Voz")
print(f"🔊 TTS: Activado")
print(f"📊 Logs: Activados")
print("="*70)



✅ SISTEMA COMPLETAMENTE OPERATIVO
🐑 Rebaño inicial: 500 ovejas
📍 Ubicaciones: 7
🏷️ Razas: 5 (Merina, Manchega, Suffolk, Dorper, Corriedale)
🎤 Modos: Texto y Voz
🔊 TTS: Activado
📊 Logs: Activados


# Pruebas

In [28]:
# PRUEBA 1: Agrupa todas las Suffolk en la nave
resultado1 = ordenar("Agrupa todas las Suffolk en la nave")



🐑 SIMULADOR DE PASTOREO INTELIGENTE

⌨️ Modo: ENTRADA POR TEXTO
📝 Orden recibida: 'Agrupa todas las Suffolk en la nave'

🔍 FASE 1: ANÁLISIS DE LA ORDEN

🔍 ANÁLISIS DE LA ORDEN
📝 Texto: Agrupa todas las Suffolk en la nave

🎯 Extracción:
   • Acción: agrupa
   • Raza: Suffolk
   • Cantidad: todas
   • Origen: Ubicación actual
   • Destino: nave

✅ Información completa - Lista para ejecutar

✅ FASE 2: CONFIRMACIÓN
📢 Voy a realizar la acción: agrupa

🔊 Sistema dice: 'Voy a realizar la acción: agrupa'



⚙️ FASE 3: EJECUCIÓN

✅ Se han movido 92 ovejas Suffolk a nave
🔊 Interjección: ¡Arre hacia allá! ¡Hacia nave!

🐑 Interjección de pastor: ¡Arre hacia allá! ¡Hacia nave!



🔊 Sistema dice: 'Se han movido 92 ovejas Suffolk a nave'



📊 FASE 4: ESTADO ACTUALIZADO DEL REBAÑO

🐑 ESTADO ACTUAL DEL REBAÑO

📍 CORRAL NORTE: 98 ovejas
   • Merina: 98

📍 PRADERA ESTE: 260 ovejas
   • Manchega: 87
   • Dorper: 73
   • Corriedale: 100

📍 NAVE: 92 ovejas
   • Suffolk: 92

📍 COBERTIZO: 20 ovejas
   • Corriedale: 20

📍 ORDEÑO: 30 ovejas
   • Manchega: 30

🐏 TOTAL REBAÑO: 500 ovejas


In [29]:
# PRUEBA 2: Separa 20 Corriedale al cobertizo
resultado2 = ordenar("Separa 20 Corriedale al cobertizo")



🐑 SIMULADOR DE PASTOREO INTELIGENTE

⌨️ Modo: ENTRADA POR TEXTO
📝 Orden recibida: 'Separa 20 Corriedale al cobertizo'

🔍 FASE 1: ANÁLISIS DE LA ORDEN

🔍 ANÁLISIS DE LA ORDEN
📝 Texto: Separa 20 Corriedale al cobertizo

🎯 Extracción:
   • Acción: separa
   • Raza: Corriedale
   • Cantidad: 20
   • Origen: Ubicación actual
   • Destino: cobertizo

✅ Información completa - Lista para ejecutar

✅ FASE 2: CONFIRMACIÓN
📢 Voy a realizar la acción: separa

🔊 Sistema dice: 'Voy a realizar la acción: separa'



⚙️ FASE 3: EJECUCIÓN

✅ Se han separado 20 ovejas Corriedale hacia cobertizo
🔊 Interjección: ¡Pss, fuera, fuera!

🐑 Interjección de pastor: ¡Pss, fuera, fuera!



🔊 Sistema dice: 'Se han separado 20 ovejas Corriedale hacia cobertizo'



📊 FASE 4: ESTADO ACTUALIZADO DEL REBAÑO

🐑 ESTADO ACTUAL DEL REBAÑO

📍 CORRAL NORTE: 98 ovejas
   • Merina: 98

📍 PRADERA ESTE: 240 ovejas
   • Manchega: 87
   • Dorper: 73
   • Corriedale: 80

📍 NAVE: 92 ovejas
   • Suffolk: 92

📍 COBERTIZO: 40 ovejas
   • Corriedale: 40

📍 ORDEÑO: 30 ovejas
   • Manchega: 30

🐏 TOTAL REBAÑO: 500 ovejas


In [30]:
# PRUEBA 3: Lleva 15 Dorper de la pradera oeste al ordeño
resultado3 = ordenar("Lleva 15 Dorper de la pradera oeste al ordeño")



🐑 SIMULADOR DE PASTOREO INTELIGENTE

⌨️ Modo: ENTRADA POR TEXTO
📝 Orden recibida: 'Lleva 15 Dorper de la pradera oeste al ordeño'

🔍 FASE 1: ANÁLISIS DE LA ORDEN

🔍 ANÁLISIS DE LA ORDEN
📝 Texto: Lleva 15 Dorper de la pradera oeste al ordeño

🎯 Extracción:
   • Acción: lleva
   • Raza: Dorper
   • Cantidad: 15
   • Origen: pradera oeste
   • Destino: ordeño

✅ Información completa - Lista para ejecutar

✅ FASE 2: CONFIRMACIÓN
📢 Voy a realizar la acción: lleva

🔊 Sistema dice: 'Voy a realizar la acción: lleva'



⚙️ FASE 3: EJECUCIÓN

❌ No hay ovejas Dorper disponibles en pradera oeste.

🔊 Sistema dice: 'No hay ovejas Dorper disponibles en pradera oeste.'



📊 FASE 4: ESTADO ACTUALIZADO DEL REBAÑO

🐑 ESTADO ACTUAL DEL REBAÑO

📍 CORRAL NORTE: 98 ovejas
   • Merina: 98

📍 PRADERA ESTE: 240 ovejas
   • Manchega: 87
   • Dorper: 73
   • Corriedale: 80

📍 NAVE: 92 ovejas
   • Suffolk: 92

📍 COBERTIZO: 40 ovejas
   • Corriedale: 40

📍 ORDEÑO: 30 ovejas
   • Manchega: 30

🐏 TOTAL REBAÑO: 500 ovejas


In [31]:
# PRUEBA 4: Traslada bastantes Merina al corral sur
resultado4 = ordenar("Traslada bastantes Merina al corral sur")



🐑 SIMULADOR DE PASTOREO INTELIGENTE

⌨️ Modo: ENTRADA POR TEXTO
📝 Orden recibida: 'Traslada bastantes Merina al corral sur'

🔍 FASE 1: ANÁLISIS DE LA ORDEN

🔍 ANÁLISIS DE LA ORDEN
📝 Texto: Traslada bastantes Merina al corral sur

🎯 Extracción:
   • Acción: traslada
   • Raza: Merina
   • Cantidad: bastantes
   • Origen: Ubicación actual
   • Destino: corral sur

✅ Información completa - Lista para ejecutar

✅ FASE 2: CONFIRMACIÓN
📢 Voy a realizar la acción: traslada

🔊 Sistema dice: 'Voy a realizar la acción: traslada'



⚙️ FASE 3: EJECUCIÓN

✅ Se han movido 68 ovejas Merina a corral sur
🔊 Interjección: ¡Vamos pa'llá! ¡Hacia corral sur!

🐑 Interjección de pastor: ¡Vamos pa'llá! ¡Hacia corral sur!



🔊 Sistema dice: 'Se han movido 68 ovejas Merina a corral sur'



📊 FASE 4: ESTADO ACTUALIZADO DEL REBAÑO

🐑 ESTADO ACTUAL DEL REBAÑO

📍 CORRAL NORTE: 30 ovejas
   • Merina: 30

📍 CORRAL SUR: 68 ovejas
   • Merina: 68

📍 PRADERA ESTE: 240 ovejas
   • Manchega: 87
   • Dorper: 73
   • Corriedale: 80

📍 NAVE: 92 ovejas
   • Suffolk: 92

📍 COBERTIZO: 40 ovejas
   • Corriedale: 40

📍 ORDEÑO: 30 ovejas
   • Manchega: 30

🐏 TOTAL REBAÑO: 500 ovejas


In [32]:
# PRUEBA 5: Procesar audio generado previamente
resultado5 = ordenar_voz("orden_prueba_1.mp3")



🐑 SIMULADOR DE PASTOREO INTELIGENTE

🎤 Modo: ENTRADA POR VOZ

🎤 PROCESANDO COMANDO DE VOZ (CON CORRECCIÓN ASR)
🔄 Convirtiendo audio a formato WAV...
🎤 Procesando audio...
🔍 Transcribiendo...
✅ Transcripción exitosa

📝 Texto transcrito: 'dime todas las ovejas merina al corral norte'
🔧 Corrección ASR aplicada:
   Antes:  'dime todas las ovejas merina al corral norte'
   Después: 'mueve todas las ovejas merina al corral norte'

🔍 ANÁLISIS DE LA ORDEN
📝 Texto: mueve todas las ovejas merina al corral norte

🎯 Extracción:
   • Acción: mueve
   • Raza: Merina
   • Cantidad: todas
   • Origen: Ubicación actual
   • Destino: corral norte

✅ Información completa - Lista para ejecutar

🔊 Sistema dice: 'Voy a realizar la acción: mueve'



🐑 Interjección de pastor: ¡Adelante! ¡Hacia corral norte!



🔊 Sistema dice: 'Se han movido 68 ovejas Merina a corral norte'



🐑 ESTADO ACTUAL DEL REBAÑO

📍 CORRAL NORTE: 98 ovejas
   • Merina: 98

📍 PRADERA ESTE: 240 ovejas
   • Manchega: 87
   • Dorper: 73
   • Corriedale: 80

📍 NAVE: 92 ovejas
   • Suffolk: 92

📍 COBERTIZO: 40 ovejas
   • Corriedale: 40

📍 ORDEÑO: 30 ovejas
   • Manchega: 30

🐏 TOTAL REBAÑO: 500 ovejas


In [33]:
# PRUEBA 6: Visualiza estado del rebaño tras las acciones
mostrar_estado()



🐑 ESTADO ACTUAL DEL REBAÑO

📍 CORRAL NORTE: 98 ovejas
   • Merina: 98

📍 PRADERA ESTE: 240 ovejas
   • Manchega: 87
   • Dorper: 73
   • Corriedale: 80

📍 NAVE: 92 ovejas
   • Suffolk: 92

📍 COBERTIZO: 40 ovejas
   • Corriedale: 40

📍 ORDEÑO: 30 ovejas
   • Manchega: 30

🐏 TOTAL REBAÑO: 500 ovejas


In [34]:
# PRUEBA 7: Visualiza el log de acciones realizadas
mostrar_log_acciones()



📜 REGISTRO DE ACCIONES (LOG DE TRAZABILIDAD)

📊 Total de acciones ejecutadas: 9
🐑 Total de ovejas movidas: 488

📋 Últimas 10 acciones realizadas:


,timestamp,accion,raza,cantidad,origen,destino,ovejas_movidas
0,2025-11-19 16:44:20,lleva,Merina,15,pradera este,corral norte,15
1,2025-11-19 16:47:28,agrupa,Suffolk,todas,pradera este,nave,92
2,2025-11-19 16:47:28,separa,Corriedale,20,pradera este,cobertizo,20
3,2025-11-19 16:47:28,lleva,Manchega,30,pradera este,ordeño,30
4,2025-11-19 16:52:49,mueve,Merina,todas,pradera este,corral norte,83
5,2025-11-19 17:00:27,agrupa,Suffolk,todas,nave,nave,92
6,2025-11-19 17:00:51,separa,Corriedale,20,pradera este,cobertizo,20
7,2025-11-19 17:01:31,traslada,Merina,bastantes,corral norte,corral sur,68
8,2025-11-19 17:01:46,mueve,Merina,todas,corral sur,corral norte,68



📈 ESTADÍSTICAS DE USO

🎯 Acciones más frecuentes:
accion
lleva       2
agrupa      2
separa      2
mueve       2
traslada    1
Name: count, dtype: int64

🐏 Razas más movidas:
raza
Merina        4
Suffolk       2
Corriedale    2
Manchega      1
Name: count, dtype: int64

📍 Destinos más usados:
destino
corral norte    3
nave            2
cobertizo       2
ordeño          1
corral sur      1
Name: count, dtype: int64


,timestamp,accion,raza,cantidad,origen,destino,resultado,ovejas_movidas
0,2025-11-19 16:44:20,lleva,Merina,15,pradera este,corral norte,"{'exito': True, 'mensaje': 'Se han movido 15 o...",15
1,2025-11-19 16:47:28,agrupa,Suffolk,todas,pradera este,nave,"{'exito': True, 'mensaje': 'Se han movido 92 o...",92
2,2025-11-19 16:47:28,separa,Corriedale,20,pradera este,cobertizo,"{'exito': True, 'mensaje': 'Se han separado 20...",20
3,2025-11-19 16:47:28,lleva,Manchega,30,pradera este,ordeño,"{'exito': True, 'mensaje': 'Se han movido 30 o...",30
4,2025-11-19 16:52:49,mueve,Merina,todas,pradera este,corral norte,"{'exito': True, 'mensaje': 'Se han movido 83 o...",83
5,2025-11-19 17:00:27,agrupa,Suffolk,todas,nave,nave,"{'exito': True, 'mensaje': 'Se han movido 92 o...",92
6,2025-11-19 17:00:51,separa,Corriedale,20,pradera este,cobertizo,"{'exito': True, 'mensaje': 'Se han separado 20...",20
7,2025-11-19 17:01:31,traslada,Merina,bastantes,corral norte,corral sur,"{'exito': True, 'mensaje': 'Se han movido 68 o...",68
8,2025-11-19 17:01:46,mueve,Merina,todas,corral sur,corral norte,"{'exito': True, 'mensaje': 'Se han movido 68 o...",68


In [35]:
# PRUEBA 8: Exporta el log a CSV para entregar
exportar_log()


💾 Log exportado como 'log_acciones_pastoreo.csv'
